In [5]:
import pandas as pd
import numpy as np

Import data

In [6]:
dfAir = pd.read_csv('source/AIR_QUALITY/world_air_quality_with_locations.csv')

In [7]:
dfAir.head()

,Unnamed: 0,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label,Lat,Long
0,0,CL,Catemu,Santa Margarita,"-32.776573, -70.938144",PM2.5,Chile - SINCA,µg/m³,0.00,2017-06-28T17:00:00+04:00,Chile,-32.776573,-70.938144
1,1,CL,Coyhaique,Coyhaique II,"-45.579045667164, -72.049964505825",SO2,Chile - SINCA,µg/m³,16.63,2023-08-09T11:00:00+04:00,Chile,-45.579046,-72.049965
2,2,CL,Calama,Colegio Pedro Vergara Keller,"-22.44283906304, -68.932546346863",PM10,Chile - SINCA,µg/m³,69.00,2023-08-09T11:00:00+04:00,Chile,-22.442839,-68.932546
3,3,CL,Coquimbo,La Serena,"-29.933006518748, -71.261966365967",PM2.5,Chile - SINCA,µg/m³,21.00,2023-08-09T11:00:00+04:00,Chile,-29.933007,-71.261966
4,4,CL,Talagante,Talagante,"-33.673752071375, -70.953064737434",NO2,Chile - SINCA,µg/m³,134.80,2019-12-05T21:00:00+04:00,Chile,-33.673752,-70.953065


From the first glimps 'Coordinates' column is just a concatenation of 'Lat' and 'Long', therefore we probably should drop one of them. 
We first check which ('Coordinates' or pair of 'Lat' and 'Long') contains more precise data and then drop redundant.

In [8]:
dfAir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43450 entries, 0 to 43449
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     43450 non-null  int64  
 1   Country Code   43450 non-null  object 
 2   City           26744 non-null  object 
 3   Location       43448 non-null  object 
 4   Coordinates    43231 non-null  object 
 5   Pollutant      43450 non-null  object 
 6   Source Name    43450 non-null  object 
 7   Unit           43450 non-null  object 
 8   Value          43450 non-null  float64
 9   Last Updated   43450 non-null  object 
 10  Country Label  43335 non-null  object 
 11  Lat            43231 non-null  float64
 12  Long           43231 non-null  float64
dtypes: float64(3), int64(1), object(9)
memory usage: 4.3+ MB


In [9]:
# Get latitude and longtitude from the Coordinates column
lat_long = dfAir['Coordinates'].str.split(',', expand=True)
lat_long.columns=['coor_lat', 'coor_long']
lat_long.head()


,coor_lat,coor_long
0,-32.776573,-70.938144
1,-45.579045667164,-72.049964505825
2,-22.44283906304,-68.932546346863
3,-29.933006518748,-71.261966365967
4,-33.673752071375,-70.953064737434


In [10]:
dfAir = pd.concat([dfAir, lat_long], axis=1)
dfAir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43450 entries, 0 to 43449
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     43450 non-null  int64  
 1   Country Code   43450 non-null  object 
 2   City           26744 non-null  object 
 3   Location       43448 non-null  object 
 4   Coordinates    43231 non-null  object 
 5   Pollutant      43450 non-null  object 
 6   Source Name    43450 non-null  object 
 7   Unit           43450 non-null  object 
 8   Value          43450 non-null  float64
 9   Last Updated   43450 non-null  object 
 10  Country Label  43335 non-null  object 
 11  Lat            43231 non-null  float64
 12  Long           43231 non-null  float64
 13  coor_lat       43231 non-null  object 
 14  coor_long      43231 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 5.0+ MB


In [11]:
for colname in ('coor_lat','coor_long'):
    dfAir[colname] = pd.to_numeric(dfAir[colname], errors='coerce')

dfAir[['coor_lat','coor_long']].head()

,coor_lat,coor_long
0,-32.776573,-70.938144
1,-45.579046,-72.049965
2,-22.442839,-68.932546
3,-29.933007,-71.261966
4,-33.673752,-70.953065


In [12]:
# check whether anything is missing after all
dfAir[((dfAir['Lat']!=dfAir['coor_lat']) | (dfAir['Long']!=dfAir['coor_long'])) & ~dfAir['Coordinates'].isna()].shape

(0, 15)

In [13]:
# check whether there is any diff
dfAir[((dfAir['Lat']!=dfAir['coor_lat']) | (dfAir['Long']!=dfAir['coor_long'])) & ~dfAir['Coordinates'].isna()].shape

(0, 15)

Visibly as string df showed better precision in 'Coordinates' column. In fact we can see that they are the same. So we drop redundant to make df more simpler.
We are not dropping any information from the set, only virtually duplicated columns. 

In [14]:
# re-arrange order and drop redundant columns
dfAir = dfAir[[ #'Unnamed: 0',
                'Country Code', 'City', 'Location',
                #'Coordinates',
                'Pollutant', 'Source Name', 'Unit', 'Value', 'Last Updated',
                'Country Label', 'Lat', 'Long']]
dfAir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43450 entries, 0 to 43449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Country Code   43450 non-null  object 
 1   City           26744 non-null  object 
 2   Location       43448 non-null  object 
 3   Pollutant      43450 non-null  object 
 4   Source Name    43450 non-null  object 
 5   Unit           43450 non-null  object 
 6   Value          43450 non-null  float64
 7   Last Updated   43450 non-null  object 
 8   Country Label  43335 non-null  object 
 9   Lat            43231 non-null  float64
 10  Long           43231 non-null  float64
dtypes: float64(3), object(8)
memory usage: 3.6+ MB


In [18]:
dfAir.nunique()

Country Code       125
City              2412
Location         10201
Pollutant           10
Source Name        143
Unit                 2
Value            10664
Last Updated      3924
Country Label      112
Lat              11044
Long             11086
dtype: int64

It seems that there are more contry's codes than lables. Worth to check which country has more than one code and why so. 

In [25]:
# sort/further cleaning
dfAir_sorted = dfAir.sort_values(by=['Country Code', 'City', 'Location', 'Unit', 'Value'], ascending=[True,True,True,True,False])
dfAir_sorted.head(30)
dfAir_sorted['City'].value_counts().get('unused',0) 

1

Noticed that there is strange name on the city in Andora, which is called 'unused'. It only appears ones, but still not very representitive, so mb worth to replace with 'Other'

In [26]:
dfAir_sorted['City'] = dfAir_sorted['City'].replace({'unused': 'Other'})

In [27]:
dfAir_sorted.head(30)

,Country Code,City,Location,Pollutant,Source Name,Unit,Value,Last Updated,Country Label,Lat,Long
2617,AD,Escaldes-Engordany,AD0942A,CO,EEA Andorra,µg/m³,100.00,2023-05-31T08:00:00+04:00,Andorra,42.509694,1.539138
27002,AD,Escaldes-Engordany,AD0942A,O3,EEA Andorra,µg/m³,50.50,2023-05-31T08:00:00+04:00,Andorra,42.509694,1.539138
4064,AD,Escaldes-Engordany,AD0942A,PM10,EEA Andorra,µg/m³,11.00,2023-05-31T08:00:00+04:00,Andorra,42.509694,1.539138
411,AD,Escaldes-Engordany,AD0942A,PM2.5,EEA Andorra,µg/m³,7.90,2023-05-31T08:00:00+04:00,Andorra,42.509694,1.539138
27003,AD,Escaldes-Engordany,AD0942A,NO2,EEA Andorra,µg/m³,4.70,2023-05-31T08:00:00+04:00,Andorra,42.509694,1.539138
39760,AD,Escaldes-Engordany,AD0942A,NO,EEA Andorra,µg/m³,0.60,2023-05-31T07:00:00+04:00,Andorra,42.509694,1.539138
7533,AD,Escaldes-Engordany,AD0942A,SO2,EEA Andorra,µg/m³,0.20,2023-05-31T08:00:00+04:00,Andorra,42.509694,1.539138
11066,AD,Escaldes-Engordany,AD0944A,O3,EEA Andorra,µg/m³,108.70,2023-05-31T08:00:00+04:00,Andorra,42.516944,1.565250
16131,AD,Other,AD0945A,O3,EEA Andorra,µg/m³,45.80,2023-04-18T18:00:00+04:00,Andorra,42.534883,1.716986
21903,AE,Abu Al Abyad,E11+Road,CO,adairquality-ae,µg/m³,430.00,2023-07-13T12:00:00+04:00,United Arab Emirates,24.035099,53.885290


Export to csv as temp, so to work further for inferenses with a clean dataset

In [ ]:
dfAir.to_csv('temp\\AIRclean.csv', index=False)